# Декораторы

In [1]:
def foo(x):
    print(x + 1)


def decorator(foo):
    def _dec(*args, **kwargs):
        print("inside decorator")
        res = foo(*args, **kwargs)
        return res
    
    return _dec

In [2]:
foo(1)

2


In [3]:
foo = decorator(foo)
foo(1)

inside decorator
2


### В Python есть специальный синтаксис, который позволяет переместить модификацию функции ближе к моменту объявления функции:

In [4]:
@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [5]:
foo(1)

inside decorator
2


### Есть небольшая проблема:

In [6]:
foo.__name__, foo.__doc__

('_dec', None)

#### Еще "пропадает" ```__module__```
#### Что делать?

In [7]:
def decorator(foo):
    def _dec(*args, **kwargs):
        res = foo(*args, **kwargs)
        return res
    
    _dec.__name__ = foo.__name__
    _dec.__doc__ = foo.__doc__
    _dec.__module__ = foo.__module__
    
    return _dec


@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [8]:
foo.__name__, foo.__doc__

('foo', '\n    foo description\n    ')

In [9]:
foo.__module__

'__main__'

#### Но это сложно, можно проще.
#### Воспользуемся декоратором ```wraps``` из пакета ```functools``` <i>(подробнее про этот пакет чуть позже)</i>

In [10]:
from functools import wraps


def decorator(foo):
    @wraps(foo)
    def _dec(*args, **kwargs):
        res = foo(*args, **kwargs)
        return res
    
    return _dec


@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [11]:
foo.__name__, foo.__doc__

('foo', '\n    foo description\n    ')

### Декораторов может быть несколько:

In [12]:
def dec1(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside dec1")
        res = func(*args, **kwargs)
        return res
        
    return _dec


def dec2(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside dec2")
        res = func(*args, **kwargs)
        return res
        
    return _dec

In [13]:
@dec2
@dec1
def func(arg1, arg2):
    pass

In [14]:
func(1, 2)

inside dec2
inside dec1


#### Это же самое, что:

In [15]:
def func(arg1, arg2):
    pass

In [16]:
func = dec2(dec1(func))

In [17]:
func(1, 2)

inside dec2
inside dec1


### Вопрос [?]

In [18]:
flag = False


def decorator(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside _dec")
        res = func(*args, **kwargs)
        return res
    
    return func if flag else _dec


@decorator
def func():
    pass

In [19]:
func()

inside _dec


#### Что будет?

In [20]:
flag = True
func()

inside _dec


### В декоратор можно передавать аргументы:

In [31]:
def decorator_with_args(dec_argument):
    def _decorator(func):
        @wraps(func)
        def _dec(*args, **kwargs):
            print(f"inside decorator; {dec_argument}")
            res = func(*args, **kwargs)
            return res
        
        return _dec
    return _decorator

In [32]:
@decorator_with_args("hop hey lala ley")
def func(x):
    pass

In [33]:
func(None)

inside decorator; hop hey lala ley


#### Это то же самое, что:

In [27]:
def decorator_with_args(dec_argument):
    def _decorator(func):
        @wraps(func)
        def _dec(*args, **kwargs):
            print(f"inside decorator; {dec_argument}")
            res = func(*args, **kwargs)
            return res
        
        return _dec
    return _decorator

In [28]:
def func(x):
    pass

In [29]:
decorator = decorator_with_args("hop hey lala ley")  # получим тут декоратор
func = decorator(func)  # получим модифицированную функцию

In [30]:
func(None)

inside decorator; hop hey lala ley


#### А если мы хотим опциональные аргументы?

In [76]:
def decorator_with_optional_arguments(func=None, *, dec_argument="default"):
    if func is None:
        def _to_return(func):
            return decorator_with_optional_arguments(func, dec_argument=dec_argument)
#         return lambda func:
        return _to_return
    @wraps(func)
    def _dec(*args, **kwargs):
        print(f"inside decorator; {dec_argument}")
        res = func(*args, **kwargs)
        return res
    
    return _dec

In [77]:
@decorator_with_optional_arguments(dec_argument="Life is beatiful")
def func1():
    pass


func1()

inside decorator; Life is beatiful


In [36]:
@decorator_with_optional_arguments
def func2():
    pass


func2()

inside decorator; default


### Несколько примеров полезных декораторов

#### Посчитать, сколько раз выполнялась функция:

In [37]:
def profiled(func):
    @wraps(func)
    def inner(*args, **kwargs):
        inner.ncalls += 1
        return func(*args, **kwargs)
    inner.ncalls = 0
    return inner

In [38]:
@profiled
def f():
    pass


for i in range(1000):
    f()
    
    
print(f.ncalls)

1000


#### Декоратор, для вызова декорируемой функции только 1 раз

In [40]:
# TODO
def once(func):
    @wraps(func)
    def inner(*args, **kwargs):
        inner.ncalls += 1
        if inner.ncalls > 1:
            raise Exception
        else:
            return func(*args, **kwargs)
    inner.ncalls = 0
    return inner    
    #pass

* [contextlib.contextmanager](https://docs.python.org/3.5/library/contextlib.html#contextlib.contextmanager)
* [functools.lru_cache](https://docs.python.org/3.5/library/functools.html#functools.lru_cache)
* почти все в [pycontracts](https://andreacensi.github.io/contracts/)
* @classmethod и @staticmethod
* @property
* @abstractmethod

...

In [41]:
@once
def f():
    print(1)

f()
f()

1


Exception: 

# Области видимости

In [21]:
print(min)  # build-in

a = 0  # global
b = 1  # global


def g():
    a = -1  # enclosing
    b = -2  # enclosing
    
    def f():
        a = 1  # local
        b = 2  # local

        print("locals: ", locals())

        print("a in globals=", globals()["a"], sep="")
        print("b in globals=", globals()["b"], sep="")
        
    f()
    
    
g()

<built-in function min>
locals:  {'b': 2, 'a': 1}
a in globals=0
b in globals=1


### А что с присваиванием?

In [22]:
a = -1  # global


def f():
    try:
        a += 1  # local
    except UnboundLocalError:
        print("Catch UnboundLocalError")


f()

Catch UnboundLocalError


#### Можно ли как-то присвоить? Да!

In [23]:
a = -1  # global


def f():
    globals()["a"] += 1  # local


f()
print(a)

0


#### Или так:

In [24]:
a = -1  # global


def f():
    global a
    a += 1  # local
    print(a)


f()

0


#### Для ```enclosing```, соответственно, вот так вот:

In [25]:
def g():
    a = -1  # enclosing

    def f():
        nonlocal a
        a += 1  # local

    f()
    print(a)
    
    
g()

0


# Дескрипторы

### @property

In [42]:
class SomeBank:
    
    def __init__(self, start):
        self._rubles = start
        
    @property
    def rubles(self):
        return self._rubles
    
    @rubles.setter
    def rubles(self, value):
        if value > 0:
            self._rubles = value
        else:
            raise Exception('You shall not pass!')

    @rubles.deleter
    def rubles(self, value):
        del self._rubles

In [43]:
interv = SomeBank(6000)

In [44]:
interv.rubles

6000

In [45]:
interv.rubles = 4242

In [46]:
interv.rubles = -32

Exception: You shall not pass!

### \_\_get__(), \_\_set__() и \_\_delete__(), 

In [65]:
class Descr:
    def __get__(self, instance, owner):
        print(instance, owner)
        
    def __set__(self, instance, value):
        print(instance, value)
        
    def __delete__(self, instance):
        print(instance)
        
    
class A:
    attr = Descr()

In [57]:
A().attr

<__main__.A object at 0x1095ee5c0> <class '__main__.A'>


In [58]:
A.attr

None <class '__main__.A'>


In [59]:
A().attr = 42

<__main__.A object at 0x1095eed68> 42


In [60]:
A.attr = 42
A.attr

42

#### Что будет?

In [61]:
A.attr = 42

In [62]:
del A().attr

AttributeError: attr

In [63]:
del A.attr

### Хм

* `instance` -- экз. класса
* `attr` -- атрибут (который дескриптор)
* `descr = cls.\_\_dict__["attr"]` -- сам дескриптор

In [66]:
instance = A()
descr = A.__dict__["attr"]

####  Тогда:

In [67]:
A.attr
descr.__get__(None, A)

None <class '__main__.A'>
None <class '__main__.A'>


In [68]:
instance.attr
descr.__get__(instance, A)

<__main__.A object at 0x1095d66a0> <class '__main__.A'>
<__main__.A object at 0x1095d66a0> <class '__main__.A'>


In [69]:
instance.attr = 42
descr.__set__(instance, 42)

<__main__.A object at 0x1095d66a0> 42
<__main__.A object at 0x1095d66a0> 42


In [70]:
del instance.attr
descr.__delete__(instance)

### Вопрос [?]

#### Где хранить данные дескриптору?

### Еще Вопрос [?]

#### Как вы думаете, как работают методы в Python?

`types.MethodType`

* http://stupidpythonideas.blogspot.com/2013/06/how-methods-work.html
* http://igorsobreira.com/2011/02/06/adding-methods-dynamically-in-python.html

# Наследование и MRO

In [ ]:
class Parent:
    def print_hello(self):
        print(f"Hello from {self.__class__}!")
        

class Child(Parent):
    pass

In [ ]:
parent = Parent()
child = Child()

parent.print_hello()
child.print_hello()

###  Пример неочевидного множественного наследования:

In [71]:
class God(object):
    def dd(self):
        print('Me God')
        
class Adam(God):
    def dd(self):
        print('Adam')
        super().dd()
        
class Eva(God): 
    def dd(self):
        print('Eva')
        super().dd()
        
class Ramon(Adam, Eva):
    def dd(self):
        print('Ramon')
        super().dd()
        
class Gayle(Adam, Eva):
    def dd(self):
        print('Gayle')
        super().dd()
        
class Raymond(Ramon, Gayle):
    def dd(self):
        print('Raymond')
        super().dd()

class Dennis(Adam, Eva): 
    def dd(self):
        print('Dennis')
        super().dd()
        
class Sharon(Adam, Eva): 
    def dd(self):
        print('Sharon')
        super().dd()
        
class Rachel(Dennis, Sharon): 
    def dd(self):
        print('Rachel')
        super().dd()

class Mattew(Raymond, Rachel): 
    def dd(self):
        print('Rachel')
        super().dd()

In [72]:
Mattew.__mro__

(__main__.Mattew,
 __main__.Raymond,
 __main__.Ramon,
 __main__.Gayle,
 __main__.Rachel,
 __main__.Dennis,
 __main__.Sharon,
 __main__.Adam,
 __main__.Eva,
 __main__.God,
 object)

In [73]:
mattew = Mattew()

mattew.dd()

Rachel
Raymond
Ramon
Gayle
Rachel
Dennis
Sharon
Adam
Eva
Me God
